In [1]:
# activate project environment
# include these lines of code in any future scripts/notebooks
#---
import Pkg
if !haskey(Pkg.installed(), "AA228FinalProject")
    jenv = joinpath(dirname(@__FILE__()), ".") # this assumes the notebook is in the same dir
    # as the Project.toml file, which should be in top level dir of the project. 
    # Change accordingly if this is not the case.
    Pkg.activate(jenv)
end
#---

# import necessary packages
using AA228FinalProject
using POMDPs
using POMDPPolicies
using BeliefUpdaters
using ParticleFilters
using POMDPSimulators
using Cairo
using Gtk
using Random
using Printf
using QMDP
using JLD


(process:76576): Gtk-WARNING **: 11:48:13.335: Locale not supported by C library.
	Using the fallback 'C' locale.


In [4]:
p = JLD.load("QMDP_Bumper_policy.jld")
p = p["policy"]

Dict{String,Any} with 1 entry:
  "policy" => AlphaVectorPolicy{RoombaPOMDP{Bumper,Bool},RoombaAct}(RoombaPOMDP…

In [2]:
num_x_pts = 50 # e.g. 50
num_y_pts = 50 # e.g. 50
num_th_pts = 20 # e.g. 20
sspace = DiscreteRoombaStateSpace(num_x_pts,num_y_pts,num_th_pts);

In [3]:
vlist = [0, 1, 2, 3]
omlist = [-1, -0.5, 0, 0.5, 1]
aspace = vec(collect(RoombaAct(v, om) for v in vlist, om in omlist));

In [4]:
sensor = Bumper();

In [5]:
config = 3 # 1,2, or 3
m = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(config=config, sspace=sspace, aspace=aspace));

In [6]:
num_particles = 2000
resampler = LidarResampler(num_particles, LowVarianceResampler(num_particles))
# for the bumper environment
# resampler = BumperResampler(num_particles)

spf = SimpleParticleFilter(m, resampler)

v_noise_coefficient = 2.0
om_noise_coefficient = 0.5

belief_updater = RoombaParticleFilter(spf, v_noise_coefficient, om_noise_coefficient);

In [8]:
solver = QMDPSolver(max_iterations=20, tolerance=1e-3, verbose=true)
p = solve(solver, m);

[Iteration 1   ] residual:        0.1 | iteration runtime:  34554.668 ms, (      34.6 s total)
[Iteration 2   ] residual:      0.095 | iteration runtime:  88547.387 ms, (       123 s total)


InterruptException: InterruptException:

In [9]:
# run the simulation
c = @GtkCanvas()
win = GtkWindow(c, "Roomba Environment", 600, 600)
for (t, step) in enumerate(stepthrough(m, p, belief_updater, max_steps=100))
    @guarded draw(c) do widget
        
        # the following lines render the room, the particles, and the roomba
        ctx = getgc(c)
        set_source_rgb(ctx,1,1,1)
        paint(ctx)
        render(ctx, m, step)
        
        # render some information that can help with debugging
        # here, we render the time-step, the state, and the observation
        move_to(ctx,300,400)
        show_text(ctx, @sprintf("t=%d, state=%s, o=%.3f",t,string(step.s),step.o))
    end
    show(c)
    sleep(0.1) # to slow down the simulation
end

In [15]:
using JLD
JLD.save("QMDP_Bumper_policy.jld", "policy", p)